In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.animation import FFMpegWriter
from IPython.display import HTML, Video

In [3]:
def lorenz(xyz, sigma=10, rho=28, beta=8/3):

    x, y, z = xyz
        
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    
    return np.array([dx, dy, dz])

In [4]:
def create_dots(start_xyz, spread, n_dots):

    start_xyz = np.array(start_xyz)
    random_deviations = np.random.uniform(-spread, spread, size=(n_dots, 3))

    return start_xyz + random_deviations

In [5]:
def anim_lorenz(
    start_xyz, spread, n_dots, n_steps, dt=0.01, 
    cmap=cm.plasma, file_type='gif', save=False
):

    # plot parameters
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize=(9, 6))
    
    ax.set_xlim(-25, 25)
    ax.set_ylim(-35, 35)
    plt.subplots_adjust(left=0.1/3, right=2.9/3, top=0.95, bottom=0.05)

    plt.axhline(-35)
    plt.axhline(35)
    plt.axvline(-25)
    plt.axvline(25)
    ax.axis('off')

    # background frame
    frame_points = 30000
    frame_xyz = np.empty((frame_points, 3))
    frame_xyz[0] = (1, 1, 1)
    
    for point in range(frame_points-1):
        frame_xyz[point+1] = frame_xyz[point] + lorenz(frame_xyz[point]) * dt
    
    ax.scatter(
        frame_xyz[:, 0], frame_xyz[:, 1], 
        s=2, color='grey', alpha=0.1
    )

    # animated motion
    points = create_dots(start_xyz, spread, n_dots)
    norm = colors.Normalize(vmin=-30, vmax=30)

    scattering = ax.scatter(
        points[:, 0], points[:, 1], c=points[:, 2],
        cmap=cmap, norm=norm, s=8
    )

    def update(frame):
        
        nonlocal points
        points = points + np.array([lorenz(point) for point in points]) * dt

        scattering.set_offsets(points[:, :2])
        scattering.set_array(points[:, 2])

        return scattering,

    anim = FuncAnimation(fig, update, frames=n_steps, blit=True)

    fps = n_steps / 10
    
    if file_type == 'gif':
        if save:
            anim.save(
                f'{start_xyz}_{n_dots}.gif', 
                writer=PillowWriter(fps=fps)
            )
            
        plt.close(fig)
        return HTML(anim.to_jshtml())

    elif file_type == 'mp4':
        title = f'{start_xyz}_{n_dots}.mp4'
        anim.save(
            title, writer=FFMpegWriter(fps=fps, codec='libx264')
        )
        
        plt.close(fig)
        return Video(title, embed=True)